In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing Relevant Libraries

In [ ]:
import torch
#import fairseq
import torchaudio
#from fairseq.models.wav2vec import Wav2VecModel
from pydub import AudioSegment
import os
import gc

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
sound = AudioSegment.from_file('../input/audio-file/audio (1).wav', "wav")

## We are handling long audio files, so best way is to break it into smaller chunks. I am using Audiosegemnt class of Pydub library

In [ ]:
t1 = 0
count = 0
#tensor_list = []
folder = './'
for i in range((int(len(sound)/100)), int(len(sound)), (int(len(sound)/100))):
    section = sound[t1:i]
    filepath = folder + str(count+1) + '.wav'
    section.export(filepath, format="wav")
    t1=i
    count+=1

## Huggingface transformers wav2vec model

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
model.to(device)

In [ ]:
folder = './'
file_list = os.listdir(folder)
file_list.remove('__notebook_source__.ipynb')

In [ ]:
def transcribe(audio_file_list, processor,device,model):
    text_list = []
    for file in audio_file_list:
        path = './'+ file
        input_values = processor(torchaudio.load(path)[0],sampling_rate=16000, padding=True, return_tensors = "pt").input_values
        input_values = input_values.squeeze().to(device)
        logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.decode(predicted_ids[0])
        text_list.append(transcription)
    return ' '.join(text_list)

In [ ]:
transcribe(file_list, processor, device, model)